This first when i training the Causal Chat model, so for do it get instruction i followed [official instructions](https://huggingface.co/docs/transformers/tasks/language_modeling)

In [1]:
class Config: 
    preset = "doublecringe123/pygmalion-dialogsum-empathetic_dialogues_llm"
    generation_params = {
#         "num_beams": 3, 
#         "do_sample": True, 
#         "temperature": 0.9,
#         "max_new_tokens": 50, 
#         "use_cache": True, 
#         "pad_token_id": 50256,
    }
    
    batch_size = 4
    epochs = 1
    train_params = {
        'save_strategy':'steps',
        'save_safetensors' : True,
        'push_to_hub': True, 
        "save_steps" :500, 
        'learning_rate':2e-5,
        'per_device_train_batch_size':batch_size,
        'per_device_eval_batch_size':batch_size,
        'weight_decay':0.01,
        'num_train_epochs':epochs,
    }
    
    
    save_path = "pygmalion-dialogsum-empathetic_dialogues_llm"
    
cfg = Config()

In [2]:
! pip install -q --upgrade pip
! pip install -q transformers[torch]
! pip install -q datasets==2.17.0 peft

def v_check(): 
    import datasets 
    import transformers
    return datasets.__version__, transformers.__version__

v_check()

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 16.0.0 which is incompatible.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.1.5 which is incompatible.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.4.0 which

('2.17.0', '4.38.2')

In [3]:
from kaggle_secrets import UserSecretsClient
secret_label = "HF_TOKEN"
secret_value = UserSecretsClient().get_secret(secret_label)

import os
os.environ["HF_TOKEN"] = secret_value

try: 
    import wandb
    wandb.init(mode='disabled')
except: 
    ...

# Load, Preprocess Dataset

In [4]:
from datasets import load_dataset 

dataset = load_dataset("doublecringe123/dataset_for_finetune_pygmalion")

Generating train split:   0%|          | 0/63986 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6540 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8094 [00:00<?, ? examples/s]

# Load Model

In [5]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

config = PeftConfig.from_pretrained(cfg.preset)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model = PeftModel.from_pretrained(model, cfg.preset, trainable=True)

model.print_trainable_parameters() 

adapter_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.44G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/717 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/205k [00:00<?, ?B/s]

trainable params: 51,200 || all params: 2,651,358,720 || trainable%: 0.0019310853568694016


# Tokenize Dataset

In [6]:
def tokenize_ds(item_batch):
    return tokenizer([item for item in item_batch['d']])

tokenized_dataset = dataset.map(tokenize_ds, batched=True, num_proc=4, remove_columns=dataset['train'].column_names)

Map (num_proc=4):   0%|          | 0/63986 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/6540 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/8094 [00:00<?, ? examples/s]

In [7]:
block_size = 128


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_dataset = tokenized_dataset.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/63986 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/6540 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/8094 [00:00<?, ? examples/s]

In [8]:
from transformers import DataCollatorForLanguageModeling
from accelerate import DataLoaderConfiguration

dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

2024-04-21 09:00:22.688313: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-21 09:00:22.688424: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-21 09:00:22.838210: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Train Model 

In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=cfg.save_path,
    evaluation_strategy="epoch",
    **cfg.train_params
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
#     evaluation takes too many time, so i will use smaller size of evaluation set
    eval_dataset=lm_dataset["test"].select(range(50)),
    data_collator=data_collator,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.187800,4.051095


TrainOutput(global_step=18988, training_loss=3.196529827997864, metrics={'train_runtime': 20028.0798, 'train_samples_per_second': 3.792, 'train_steps_per_second': 0.948, 'total_flos': 1.4684126038327296e+17, 'train_loss': 3.196529827997864, 'epoch': 1.0})

In [10]:
print(f"({cfg.epochs} epoch) Saving finnal model...")    

model.save_pretrained(cfg.save_path)
model.push_to_hub(cfg.save_path, commit_message = f"First version {cfg.epochs} epochs")

(1 epoch) Saving finnal model...


README.md:   0%|          | 0.00/5.10k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/205k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/doublecringe123/pygmalion-dialogsum-empathetic_dialogues_llm/commit/214e7f25aa14cd2d84365715c089f2314c9031aa', commit_message='First version 1 epochs', commit_description='', oid='214e7f25aa14cd2d84365715c089f2314c9031aa', pr_url=None, pr_revision=None, pr_num=None)